#  Homework 2
## Regularization, Logistic Regression


**Harvard University**<br/>
**Summer 2019**<br/>
**Instructor**: Pavlos Protopapas <br>
**Homework prepared by:** David Sondak and Andrea Porelli

---

### INSTRUCTIONS

- Submisson instructions:
  - Submit your Jupyter notebook to the teaching staff in an email.
  - The email should have the following subject:  gec-summer-2019 HW 2 "Your Name"
    - e.g. gec-summer2019 HW2 David Sondak
    
    **Homeworks with the wrong subject line will recieve a 0.**
  - The due date is Friday, August 23rd at 23:59 PM EDT.  Please note the time-zone!
    ** No late days!**

- Restart the kernel and run the whole notebook again before you submit. 
---

##### Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
%matplotlib inline

# Part 1 [25 pts]: Regularization
## Problem Description:  Forecasting Bike Sharing Usage
Part 1 of this homework (HW2) continues Part 2 of HW1.  You will continue to build regression models for the Capital Bikeshare program in Washington D.C.  Part 2 of HW1 contains a detailed description of the dataset in case you need a reminder.

**Important:** The present problem (Part 1 of HW2) assumes that you have already pre-processed the dataset.  If you did not save the processed data from subparts $2.1$ and $2.2$ in HW1, then you must redo the pre-processing.  Specifically>
* You must convert categorical attributes into multiple binary attributes using one-hot encoding.
* Scale each continuous predictor to have zero mean and a standard deviation of  $1$.
See subpart $2.2$ in HW1 for even more details.

##### Goals of this Part
You will fit a multilinear regression model using using lasso and ridge regression and compare their performance.

In [14]:
dfbike_train = pd.read_csv("BikeShare_train_Pre_Processed")
dfbike_test = pd.read_csv("BikeShare_test_Pre_Processed")
dfbike_train

,Unnamed: 0,Spring,Summer,Fall,Jan,Feb,Mar,Apr,May,Jun,...,Sat,is_workingday,weather_1.0,weather_2.0,weather_3.0,temp,atemp,humidity,windspeed,TotalCount
0,0,0,1,0,0,0,0,0,1,0,...,0,1,0,1,0,0.623798,0.650106,0.920664,-0.928758,6073.0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,-0.180310,-0.054759,0.696852,-0.213502,6606.0
2,2,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,0.802489,0.851495,-0.448383,0.803926,7363.0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,-1.520492,-1.565182,-0.332113,-0.269099,2431.0
4,4,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0.534453,0.348021,1.975789,-1.199027,1996.0
5,5,0,1,0,0,0,0,0,1,0,...,0,1,0,1,0,0.623798,0.650106,0.441062,1.940601,4451.0
6,6,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,-0.895074,-0.860318,-2.095460,2.006654,5315.0
7,7,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,0.534453,0.650106,-0.884385,1.002672,4891.0
8,8,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0.534453,0.650106,-0.922167,0.724530,8714.0
9,9,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,1.159871,1.052885,-0.099581,-0.269202,7347.0


## Part 1.1 [25 pts]:  Ridge and Lasso Regression

### Part 1.1.1 [6 pts]:  Fit Linear, Ridge, and Lasso Regression Models
Use the following regression techniques to fit linear models to the training set:
- Ridge regression
- Lasso regression

Choose the regularization parameter $\lambda$ from the set $\{0.001, 0.005,..., 50, 100\}$ using cross-validation.

#### Hints
- You may use `sklearn`'s `LinearRegression`, `RidgeCV` and `LassoCV` classes to implement Linear, Ridge, and Lasso regression. These classes automatically perform cross-validation to tune the parameter $\lambda$ from a given range of values.
- You may use the `plt.errorbar` function to plot confidence bars for the average $R^2$ scores.

#### Deliverables
Your code should be contained in a Jupyter notebook cell.  An appropriate level of comments is necessary.  Your code should run and output the required outputs described below.

#### Required Outputs
- Print the train and test $R^2$ scores for each model

In [15]:
#Linear Model
# Add column of ones to x matrix
xtrain_vals = sm.add_constant(x_train.values)
ytrain_vals = y_train.values
xtest_vals = sm.add_constant(x_test.values)
ytest_vals = y_test.values
# Create model for linear regression
model = sm.OLS(ytrain_vals, xtrain_vals)
# Fit model
fitted_model = model.fit()
a = fitted_model.summary()
def multiple_linear_regression_score(x_test, y_test):
    # Compute predicted labels
    
    y_predicted = fitted_model.predict(x_test)
    
    # Evaluate sqaured error, against target labels
    # sq_error = \sum_i (y[i] - y_pred[i])^2
    sq_er = np.sum(np.square(y_test - y_predicted))
    
    # Evaluate squared error for a predicting the mean value, against target labels
    # variance = \sum_i (y[i] - y_mean)^2
    y_mean = np.mean(y_test)
    y_variance = np.sum(np.square(y_test - y_mean))
    
    # Evaluate R^2 score value
    r_squared = 1.0 - sq_er / y_variance

    return r_squared, y_predicted
r_2,predicted_y = multiple_linear_regression_score(xtest_vals, ytest_vals)
print("The R square in LinearRegression is:",r_2)

NameError: name 'x_train' is not defined

In [ ]:
#Ridge

In [ ]:
#Lasso

### 1.1.2 [4 pts]:  Discuss your results by answering the following questions:
1. [1 pt] How do the estimated coefficients compare to or differ from the coefficients estimated by a plain linear regression (without regularzation)?

  *Your answer here.*

2. [1 pt] Is there a difference between coefficients estimated by the two shrinkage methods? If so, give an explantion for the difference.

  *Your answer here.*

3. [1 pt] List the predictors that are assigned a coefficient value close to 0 (say < 1e-10) by the two methods.

  *Your answer here.*

4. [1 pt] Is there a difference in the way Ridge and Lasso regression assign coefficients to the predictors `temp` and `atemp`? If so, explain the reason for the difference.

  *Your answer here.*

### 1.1.3 [12 pts]:  Analyze the performance of the two regularization methods for different training sample sizes
It would be nice to assess the performance of the regularization methods to figure out which one works better.  To do this, you will execute the following steps:
- Generate random samples of sizes 100, 150, ..., 400 from the training set.
- Do a random split of the data at each sample size.  You will do this 10 times.
- Fit linear, Ridge and Lasso models on each training set.
- Calculate the mean $R^2$ score and it's standard deviation (SD) for each model.
- Compare the performance.

#### Hints
- You may use the following code to draw a random sample of a specified size from the training set:

```python
#--------  sample
# A function to select a random sample of size k from the training set
# Input: 
#      x (n x d array of predictors in training data)
#      y (n x 1 array of response variable vals in training data)
#      k (size of sample) 
# Return: 
#      chosen sample of predictors and responses

def sample(x, y, k):
    n = x.shape[0] # No. of training points
    
    # Choose random indices of size 'k'
    subset_ind = np.random.choice(np.arange(n), k.astype(int))
    
    # Get predictors and reponses with the indices
    x_subset = x[subset_ind, :]
    y_subset = y[subset_ind]
    
    return (x_subset, y_subset)
```

#### Deliverables
Your code should be contained in a Jupyter notebook cell.  An appropriate level of comments is necessary.  Your code should run and output the required outputs described below.

#### Required Outputs
- Fit Linear, Ridge and Lasso regression models to each of the generated samples. 
  - In each case, compute the $R^2$ score for the model on the training sample on which it was fitted and on the test set.
- Repeat the above experiment for $10$ random trials
  - Compute the average train and test $R^2$ across the trials for each training sample size.
  - Compute the standard deviation (SD) in each case.
- Make a plot of the mean training $R^2$ scores for the Linear, Ridge and Lasso regression methods as a function of the training sample size.
- Show a confidence interval for the mean scores extending from **mean - SD** to **mean + SD**.
- Make a similar plot for the test $R^2$ scores.

### 1.1.4 [3 pts]:  Discuss your results by answering the following questions:
1. [1 pt] How do the training and test $R^2$ scores compare for the three methods? Give an explanation for your observations.

  *Your answer here.*

2. [1 pt] How do the confidence intervals for the estimated $R^2$ change with training sample size?

  *Your answer here.*

3. [1 pt] Based on the plots, which of the three methods would you recommend when one needs to fit a regression model using a small training sample?

  *Your answer here.*


---

# Part 2 [22 pts]:  Logistic Regression

## Problem Description:  Cancer Classification from Gene Expressions

In this part of the assignment, you will build a classification model to distinguish between two related classes of cancer, acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML), using gene expression measurements. 

The data set is provided in the file `dataset_hw2.csv`. 
* Rows in the file:  tumor tissue samples from a patient with one of the two forms of leukemia. 
* The first column contains the cancer type, with 0 indicating the ALL class and 1 indicating the AML class. 
* Columns 2-7130 contain expression levels of 7129 genes recorded from each tissue sample. 

##### Goals of this part
Use logistic regression to build a classification model for this data set.

## Part 2.1 [6 pts]: Data Exploration

### Part 2.1.1 [2 pts]:  Pre-processing
* Split  the observations into an approximate 50-50 train-test split.  Below is some code to do this for you (we want to make sure everyone has the same splits).
```python
np.random.seed(9001)
df = pd.read_csv('dataset_hw2.csv')
msk = np.random.rand(len(df)) < 0.5
data_train = df[msk]
data_test = df[~msk]
```
* Take a peak at your training set: you should notice the severe differences in the measurements from one gene to the next (some are negative, some hover around zero, and some are well into the thousands).  To account for these differences in scale and variability, normalize each predictor to vary between 0 and 1.

#### Deliverables
Your code should be contained in a Jupyter notebook cell. An appropriate level of comments is necessary. Your code should run.

#### Required Outputs
None for this section.

### Part 2.1.2 [2 pts]:  Heat Map
A convenient tool to visualize the gene expression data is a heat map. 
* Arrange the rows of the training set so that the *AML* rows are grouped together and the *ALL* rows are together. * * Generate a heat map of the data with expression values from the following genes:
  - `D49818_at`
  - `M23161_at`
  - `hum_alu_at`
  - `AFFX-PheX-5_at`
  - `M15990_at`

  **Hint:** Take a look at the `seaborn` `heatmap` method:  [`seaborn` heat map](https://seaborn.pydata.org/generated/seaborn.heatmap.html) and [seaborn-heatmap-using-pandas-dataframe](https://stackoverflow.com/questions/37790429/seaborn-heatmap-using-pandas-dataframe).

#### Deliverables
Your code should be contained in a Jupyter notebook cell. An appropriate level of comments is necessary. Your code should run and output the required outputs described below.

#### Required Outputs
1. A properly labeled heat map.
  * Properly labeled means $x$ and $y$ axes are labeled and a colorbar is included.

### Part 2.1.3 [2 pts]:  Discussion Questions

1. [1 pt] Notice that the results training set contains more predictors than observations. Do you foresee a problem in fitting a classification model to such a data set?

  *Your answer here.*

2. [1 pt] By observing the heat map, comment on which of these genes are useful in discriminating between the two classes.

  *Your answer here.*

## Part 2.2 [16 pts]:  Linear Regression vs. Logistic Regression
In this part, you will analyze the differences between using linear regression and logistic regression for classification. For this part, you will work with a single gene predictor: `D29963_at`.

### 2.2.1 [2 pts]:  Linear Regression
* Fit a simple linear regression model to the training set using the single gene predictor `D29963_at`.

#### Deliverables
Your code should be contained in a Jupyter notebook cell. An appropriate level of comments is necessary. Your code should run and output the required outputs described below.

#### Required Outputs
* Print the prediction from the linear regression model, $\widehat{y}_{\textrm{test}}$

### 2.2.2 [2 pts]:  Convert to Classification Model
The fitted linear regression model can be converted to a classification model (i.e. a model that predicts one of two binary labels 0 or 1) by classifying patients with predicted score greater than 0.5 into the `ALL` type (class 1), and the others into the `AML` type (class 0).

* Evaluate the classification accuracy ($1$ - misclassification rate) of the obtained classification model on both the training and test sets.

#### Deliverables
Your code should be contained in a Jupyter notebook cell. An appropriate level of comments is necessary. Your code should run and output the required outputs described below.

#### Required Outputs
* Print the train accuracy and test accuracy.  For example:
```python
Train accuracy:  0.78125
Test accuracy:  0.731707317073
```

### 2.2.3 [2 pts]:  Logistic Regression
Fit a simple logistic regression model to the training set.
**Hint:**:  Remember, you need to set the regularization parameter for `sklearn`'s logistic regression function to be a very large value in order not to regularize (use 'C=100000').

#### Deliverables
Your code should be contained in a Jupyter notebook cell. An appropriate level of comments is necessary. Your code should run and output the required outputs described below.

#### Required Outputs
* Print the train and test accuracy.  For example:
```python
Train accuracy:  0.75
Test accuracy:  0.80487804878
```

### 2.2.4 [5 pts]:  Visualize Results
* Plot the quantitative output from linear regression model and the probabilistic output from the logistic regression model (on the training set points) as a function of the gene predictor.
* Display the true binary response for the training set points in the same plot.

#### Deliverables
Your code should be contained in a Jupyter notebook cell. An appropriate level of comments is necessary. Your code should run and output the required outputs described below.

#### Required Outputs
1. One figure containing three plots side-by-side:
  a. The left plot should be the `OLS` prediction
  b. The middle plot should be the logistic prediction
  c. The right plot should be the true binary response
2. Make the same plot for the test set points.
3. Be sure to properly annotate your figures (label $x$ and $y$ axes, put a title, put a legend)

**Hint**:  For side-by-side plots, consider using the construct:
```python
fig, ax = plt.subplots(1,3, figsize=(10,6))
ax[0].plot(...) # left plot
ax[1].plot(...) # middle plot
ax[2].plot(...) # right plot
```

### 2.2.5 [3 pts]:  Discussion Questions
1. [1 pt] We could interpret the scores predicted by regression model (part $2.2.1$) interpreted for a patient as an estimate of the probability that the patient has the `ALL` type cancer (class 1). Is there a problem with this interpretation?

  *Your answer here.*

2. [1 pt] How does the training and test classification accuracy of the logistic regression model compare with the linear regression model?

  *Your answer here.*

3. [1 pt] Based on these plots in part $2.2.4$, does one of the models appear better suited for binary classification than the other? Explain.